In [ ]:
pip install ruptures

In [ ]:
import numpy as np
import pandas as pd
import ruptures as rpt
import matplotlib.pyplot as plt
import statsmodels.api as sm

In [ ]:
nb_actifs = 20
nb_dates = 256 * 5

idx = pd.Index(np.arange(nb_actifs))

mu = pd.Series(0.05,index=idx)
vols = pd.Series(0.2,index=idx)
correl = 0.7

covar = np.diag(vols) @ pd.DataFrame(correl * np.ones((nb_actifs,nb_actifs)) + (1-correl) * np.eye(nb_actifs),index=idx,columns=idx) @ np.diag(vols)
A = np.linalg.cholesky(covar)

rdts_observes = mu/256 + pd.DataFrame(np.random.randn(nb_dates,nb_actifs)) @ A.T / 16


In [ ]:
X=rdts_observes
y=np.ones(nb_dates)

In [ ]:
question : les points de rupture devraient-ils etre : rdt en fonction d'actif ? ou autre chose ?

In [ ]:
model = "l1"  
X_array = np.array(X)
algo = rpt.Dynp(model=model, min_size=5, jump=5).fit(X_array.T)
result = algo.predict(n_bkps=1)
print(result)
#ca renvoie des points de rupture au niveau de l'actif 15 et de l'actif 20 ??? ou au niveau de la date 15 et de la date 20 ?
#-> a priori, ça renvoie points de rupture au niveau des actif 15 et 20 (donc un seul point de rupture en fait)
# bizarre: quand on teste le code plein de fois, les points de rupture sont uniquement 5 ou 15 ou 20 

In [ ]:
segments = np.split(y, result)
print(segments)


models = []
for i, segment in enumerate(segments):
    X_segment = np.arange(len(segment))
    model = sm.OLS(segment, X_segment).fit()
    models.append(model)


# Prédire les valeurs pour chaque segment
predicted_values = np.concatenate([model.predict(X_segment) for model, X_segment in zip(models, [np.arange(len(segment)) for segment in segments])])
print(predicted_values)
print(len(predicted_values))

plt.plot(X, y, label='Données')
plt.plot(X, predicted_values, label='Régression segmentée', linestyle='dashed', color='red')
plt.scatter(result, [y[idx] for idx in breakpoints], color='red', marker='x', label='Points de rupture')
plt.title('Régression segmentée sans intercept avec points de rupture')
plt.legend()
plt.show()